In [1]:
#import tools, load required data
import sys
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import re
import csv
import os
pd.set_option("display.max_rows",300)
pd.set_option("display.max_columns",100)

## Load files

In [2]:
#load all files
preDX = pd.read_csv("/home/ucsdantibiogram/notebooks/data/round2data/Diag_before_order.csv", header = None, names = ("MRN", "Visit_id", "ICD9", "ICD10", "preDX_time", "PP_YN"))
orderDX = pd.read_csv("/home/ucsdantibiogram/notebooks/data/round2data/Diag_at_order.csv", header = None, names = ("MRN", "Visit_id", "ICD9", "ICD10", "preDX_time", "PP_YN"))
medsBeforeOrder = pd.read_csv("/home/ucsdantibiogram/notebooks/data/round2data/Med_before_order.csv", header = None, names = ("MRN", "Visit_id", "Order_id", "Med_name", "order_mode", "Patient_type", "ordering_date", "start_date"))
medsAtOrder = pd.read_csv("/home/ucsdantibiogram/notebooks/data/round2data/Med_at_order.csv", header = None, names = ("MRN", "Visit_id", "Order_id", "Med_name", "order_mode", "Patient_type", "ordering_date", "start_date"))
dischargeMeds = pd.read_csv("/home/ucsdantibiogram/notebooks/data/round2data/Med_after_order.csv", header = None, names = ("MRN", "Visit_id", "Order_id", "Med_name", "order_mode", "Patient_type", "ordering_date", "start_date"))
orderInfo = pd.read_csv("/home/ucsdantibiogram/notebooks/data/round2data/Order_info.csv", header = None, names = ("MRN", "Visit_id", "Order_id", "Order_time", "line", "description", "antibiotic", "susceptibility", "sensitivity_value", "organism", "specimen_taken_time", "specimen_comments", "specimen_type", "result_date", "hosp_dept", "enc_dept", "admission_time", "specimen_source", "Discharge_Disposition", "Discharge_time"))
problem_list = pd.read_csv("/home/ucsdantibiogram/notebooks/data/round2data/Problem_List.csv", header = None, names = ("MRN", "Visit_id", "ICD9", "ICD10", "DX_time", "PP_YN", "Activity"))
demo = pd.read_csv("/home/ucsdantibiogram/notebooks/data/round2data/Demographic.csv")
social = pd.read_csv("/home/ucsdantibiogram/notebooks/data/round2data/social.csv")
dischargeDiag = pd.read_csv("/home/ucsdantibiogram/notebooks/data/round2data/discharge_diag.csv")
dischargeMeds = pd.read_csv("/home/ucsdantibiogram/notebooks/data/round2data/discharge_meds.csv", encoding = "ISO-8859-1")


In [3]:
orderInfo.loc[orderInfo['hosp_dept'].isnull(),'hosp_dept'] = orderInfo["enc_dept"]
orderInfo.drop("enc_dept", axis=1, inplace=True)

In [4]:
del demo["Unnamed: 7"]

In [5]:
orderDXCorrection = orderDX[ (~orderDX["ICD10"].isnull()) & (orderDX["PP_YN"] == "Y") ][['MRN', 'Visit_id', 'ICD10']].drop_duplicates(keep = "first")
orderDXCorrection['ICD10'] = orderDXCorrection["ICD10"].str.split(',', expand = True)

## One Hot Encode ICD10 and shorten to top level

In [6]:
orderDXLimited = orderDX[ (~orderDX["ICD10"].isnull())][['Visit_id', 'ICD10', 'PP_YN']].drop_duplicates(keep = "first")

#shorten ICD10
orderDXLimited["ICD10"]=orderDXLimited["ICD10"].str.split(".").str[0]
orderDXLimited.drop_duplicates(keep = "first")

#orderDXLimited['ICD10'] = orderDXLimited["ICD10"].str.split(',', expand = True)

isprincipal=orderDXLimited['PP_YN'] == 'Y'
orderDXLimitedPrincipal=orderDXLimited.loc[isprincipal]
orderDXLimitedNotPrincipal=orderDXLimited.loc[~isprincipal]

def encodeICD10(icd10list):
    
    #deal with ICD10 with commas
    a = pd.DataFrame(icd10list["ICD10"].str.split(',').tolist(), index=icd10list["Visit_id"]).stack()
    a = a.reset_index()[[0, 'Visit_id']] # var1 variable is currently labeled 0
    a.columns = ['ICD10', 'Visit_id'] # renaming var1
    a['ICD10']=a['ICD10'].str.strip()
    #hot encode ICD10
    a=pd.get_dummies(a, prefix="", prefix_sep="")#, sparse=True)
    a=a.groupby("Visit_id").sum().reset_index()
    return a

### only principal problem

In [7]:
hotEncodePrincipalICD10=encodeICD10(orderDXLimitedPrincipal)

In [8]:
hotEncodePrincipalICD10

,Visit_id,A01,A02,A03,A04,A05,A06,A07,A08,A09,A15,A17,A18,A19,A27,A31,A35,A37,A39,A40,A41,A42,A43,A46,A48,A49,A52,A53,A54,A55,A56,A57,A59,A60,A63,A64,A74,A78,A80,A81,A87,A92,A98,B00,B01,B02,B07,B08,B09,B10,...,Z36,Z37,Z38,Z39,Z3A,Z40,Z41,Z42,Z43,Z44,Z45,Z46,Z47,Z48,Z49,Z51,Z52,Z53,Z59,Z62,Z63,Z64,Z65,Z66,Z68,Z71,Z72,Z74,Z76,Z77,Z78,Z79,Z80,Z81,Z83,Z85,Z86,Z87,Z88,Z89,Z90,Z91,Z92,Z93,Z94,Z95,Z96,Z97,Z98,Z99
0,16244531,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
1,16303797,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,16447375,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,16586417,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,16624748,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,16628017,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,16719525,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,16721429,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,16738161,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,16748935,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


### all problems

In [9]:
hotEncodeAllICD10=encodeICD10(orderDXLimited)

#pd.concat([hotEncodePrincipalICD10.sum(axis=1), hotEncodeAllICD10.sum(axis=1)], axis=1)

## Pre-Process OrderInfo

### Limit OrderInfo to Organisms that Appear >100 Times

In [10]:
OILimited = orderInfo[["MRN", "Visit_id", "Order_time", "antibiotic", "susceptibility", "organism", "description", "hosp_dept"]]
pd.to_datetime( orderInfo["Order_time"])

orgValueCounts = OILimited['organism'].value_counts()
OILimited = OILimited[OILimited['organism'].isin(orgValueCounts[orgValueCounts > 100].index)]

OILimited.shape

(1070096, 8)

## One Hot Encode Antibiotics with Mapped Susceptibilities 

In [11]:
#map susceptibility values to numbers
suscmapping={'Susceptible':1,
             'Positive':1,
             'Resistant':-1, 
             'Negative': -1, 
             'Not Susceptible':-1,
             'Intermediate':-0.5,              
             'Not Defined': -0.5,         
             'Dose Dependent':-0.5}

sus = OILimited['susceptibility'].map(suscmapping)

#preprocessing turn antibiotics into columns
antibioticColumns = pd.get_dummies(OILimited['antibiotic'])
antibioticColumns=antibioticColumns.multiply(sus.values, axis=0)
OIEncoded=OILimited.drop('susceptibility', axis=1)
OIEncoded=OIEncoded.drop('antibiotic', axis=1)
OIEncoded=OIEncoded.join(antibioticColumns)

#group by visit_id (visit_id is unique and better suited than MRN) and specimen_comments (sometimes several samples are analyzed at the same time)
OICompressed = OIEncoded.groupby(['MRN',
                               'Visit_id',
                               'Order_time', 
                               'organism',
                               'description',  
                               'hosp_dept', 
                             ]).agg(sum).reset_index()

OICompressed.shape

(84260, 77)

In [12]:
suscColumns=OICompressed.columns[6:]
suscColumns=suscColumns.insert(0, 'Visit_id')
suscColumns=suscColumns.insert(1, 'organism')

susceptibilities =  OICompressed[suscColumns].copy()
susceptibilities['organism'] = susceptibilities['organism'].astype('category')

susceptibilities.shape

(84260, 73)

## One Hot Encode Organisms

In [13]:
#preprocessing turn organisms into columns

orgColumns = pd.get_dummies(OICompressed['organism'])
OICompressed=OICompressed.drop('organism', axis=1)
OIEncoded=OICompressed.join(orgColumns)
#OIEncoded

## One Hot Encode SampleDescription

In [14]:
#preprocessing turn organisms into columns
descColumns = pd.get_dummies(OICompressed['description'])
OICompressed=OICompressed.drop('description', axis=1)
OIEncoded2=OICompressed.join(descColumns)

## One Hot Encode Department 

In [15]:
#preprocessing turn organisms into columns
deptColumns = pd.get_dummies(OICompressed['hosp_dept'])
OIEncoded3=OICompressed.join(deptColumns)
OIEncoded3.shape

(84260, 348)

## Pre-Process medsAtOrder

### Limit to Antibiotics Only

In [16]:
antiSet = orderInfo["antibiotic"].str.upper().unique()
MAOLimited = medsAtOrder[["Visit_id",  "Med_name"]].copy()
MAOLimited['Med_name']=MAOLimited['Med_name'].str.split().str[0].replace('-', '/')

MAOLimited = MAOLimited[MAOLimited["Med_name"].isin( antiSet )]   
MAOLimited.rename( columns = {"Med_name": "Antis"}, inplace = True)
MAOLimited.drop_duplicates( subset = ("Visit_id", "Antis"), inplace = True)

### One Hot Encode Antis at Order

In [17]:
#pre-process MAOLimited to One Hot Encode Antis
#MAOLimited["Antis"] =  MAOLimited["Antis"].str.upper() + "|"
#MAOLimitedToEncode = MAOLimited.groupby(["Visit_id"]).agg( sum )
#hotEncodeAntis = MAOLimitedToEncode["Antis"].str.split('|', expand=True).stack()

#hotEncodeAntis

In [18]:
#one hot encode ICD10
hotEncodeAntis = pd.get_dummies(MAOLimited, prefix='At_order').groupby("Visit_id").sum().reset_index()
#del hotEncodeAntis["At_order_"]

hotEncodeAntis.shape

(30307, 53)

## Pre-Process Social and Demographic

In [19]:
social = social.replace( 'Y', 1 )
social = social.replace( 'N', 0 )

recentSocial = social.drop_duplicates( subset = "MRN", keep = "last")
recentSocial.shape

(29602, 3)

In [20]:
#shorten zip
demo['zip']=demo['zip'].str.slice(0,3)
#exclude canada and other non-US zips by mapping them to 0
demo['zip']=demo['zip'].apply(lambda x : x if (str(x).isdigit()) else 0)

# get rid of nan in race and map them to zero
demo.fillna(0, inplace=True)

In [21]:
demo = demo.replace( 'Y', 1 )
demo = demo.replace( 'N', 0 )
demo = demo.replace( 'Male', 1 )
demo = demo.replace( 'Female', -1 )
demo = demo.replace( 'nan', 0 )

demo["race"]=demo["race"].astype(str).replace("0", "Unknown (Patient cannot or refuses to declare race)")
raceColumns=pd.get_dummies(demo["race"])
demo=demo.drop('race', axis=1)
demo=demo.join(raceColumns)

recentDemo = demo.drop_duplicates( subset = "MRN", keep = "last")


## Begin Assembling Comprehensive Dataset

build similar to sklearn structure

### start with selected rows from order information

In [22]:
data=OICompressed[["MRN", "Visit_id", "Order_time"]].copy()
data["Order_time"]=pd.to_datetime(data["Order_time"])
data = data.merge( recentSocial, on = ("MRN"), how = "left")
data= data.merge( recentDemo, on = "MRN", how = "inner")
data.drop("MRN", axis=1, inplace=True)

# add source hot encoded
data = data.merge(descColumns, left_index=True, right_index=True)

# add hospital department hot encoded
data = data.merge(deptColumns, left_index=True, right_index=True)

data = data.merge(hotEncodeAllICD10, on="Visit_id", how = "left")

data = data.merge( hotEncodeAntis, how = "left", on = "Visit_id")


data.shape

(84260, 1848)

In [23]:
data

,Visit_id,Order_time,IV_drug_user_yn,male_partner_yn,Gender,age_at_order,zip,prisoner_YN,death_within_4week,American Indian or Alaska Native,Asian,Black or African American,Native Hawaiian or Other Pacific Islander,Other Race or Mixed Race,Unknown (Patient cannot or refuses to declare race),White,"AFB BLOOD CLT,BACTEC",AFB CULTURE W/STAIN,"AFB, BONE MARROW CULTURE",ANAEROBIC CULTURE,BLOOD CULTURE QUANTITATION ONLY,BLOOD CULTURE ROUTINE,"BODY SITE W/GRAM STAIN, AEROBIC ONLY","CSF CULTURE W GRAM STAIN, AEROBIC ONLY",FLUID CULTURE IN BLOOD BOTTLES,FUNGAL BLOOD CULTURE,FUNGAL BONE MARROW CUTURE,FUNGAL CULTURE,GASTROINTESTINAL PATHOGEN NUCLEIC ACID DETECTION TEST,"GENERAL AEROBIC CULTURE, OR ONLY",GENITAL CULTURE,GROUP A STREP CULTURE,GROUP B STREP CULTURE,"ID, AEROBIC",MRSA SCREEN CULTURE,ORGANISM SENSITIVITY,QUANT BRONCH CULTURE W/GRAM STAIN,QUANT TISSUE CULTURE W/GRAM STAIN,RECTAL CULTURE,RECTAL CULTURE PRE PROSTATE BIOPSY SCREEN,RESPIRATORY CULTURE W/GRAM STAIN,ROUTINE COLLECT RESPIRATORY CULTURE W/GRAM STAIN,"STERILE FLUID CULTURE W/GRAM STAIN, AEROBIC",STOOL CULTURE,TIP CULTURE,"TISSUE CULTURE W/GRAM STAIN, AEROBIC","TRANSFUSION REACTION, BLOOD PRODUCT CULTURE",URINE CULTURE,VRE SCREEN CULTURE,YEAST CULTURE,...,At_order_AMPICILLIN,At_order_AZITHROMYCIN,At_order_AZTREONAM,At_order_CEFAZOLIN,At_order_CEFEPIME,At_order_CEFOTAXIME,At_order_CEFOXITIN,At_order_CEFTAROLINE,At_order_CEFTAZIDIME,At_order_CEFTRIAXONE,At_order_CEFUROXIME,At_order_CHLORAMPHENICOL,At_order_CIPROFLOXACIN,At_order_CLARITHROMYCIN,At_order_CLINDAMYCIN,At_order_COLISTIN,At_order_CYCLOSERINE,At_order_DAPTOMYCIN,At_order_DORIPENEM,At_order_DOXYCYCLINE,At_order_ERTAPENEM,At_order_ERYTHROMYCIN,At_order_ETHAMBUTOL,At_order_ETHIONAMIDE,At_order_FLUCONAZOLE,At_order_FOSFOMYCIN,At_order_GENTAMICIN,At_order_ISONIAZID,At_order_ITRACONAZOLE,At_order_LEVOFLOXACIN,At_order_LINEZOLID,At_order_MEROPENEM,At_order_METRONIDAZOLE,At_order_MICAFUNGIN,At_order_MINOCYCLINE,At_order_MOXIFLOXACIN,At_order_NITROFURANTOIN,At_order_OFLOXACIN,At_order_OXACILLIN,At_order_PIPERACILLIN,At_order_POSACONAZOLE,At_order_PYRAZINAMIDE,At_order_RIFABUTIN,At_order_RIFAMPIN,At_order_STREPTOMYCIN,At_order_TETRACYCLINE,At_order_TIGECYCLINE,At_order_TOBRAMYCIN,At_order_VANCOMYCIN,At_order_VORICONAZOLE
0,66008241995,2015-10-23 00:00:00,0.0,0.0,-1,74,921,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,66009338937,2016-01-07 10:04:00,0.0,0.0,-1,74,921,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,66009427034,2016-01-14 09:25:00,0.0,0.0,-1,74,921,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,66009459910,2016-01-18 09:21:00,0.0,0.0,-1,74,921,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,22257231,2013-07-19 14:09:00,1.0,0.0,1,54,921,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
5,22257231,2013-07-19 14:09:

In [24]:
target=susceptibilities
target["death_within_4week"]=data["death_within_4week"]
data.drop("death_within_4week", axis=1, inplace=True)


target.shape

(84260, 74)

In [25]:
%store -z 

### Store Data

In [26]:
%store data
%store target

Stored 'data' (DataFrame)
Stored 'target' (DataFrame)


In [27]:
who

MAOLimited	 OICompressed	 OIEncoded	 OIEncoded2	 OIEncoded3	 OILimited	 antiSet	 antibioticColumns	 csv	 
data	 demo	 deptColumns	 descColumns	 dischargeDiag	 dischargeMeds	 encodeICD10	 hotEncodeAllICD10	 hotEncodeAntis	 
hotEncodePrincipalICD10	 isprincipal	 medsAtOrder	 medsBeforeOrder	 np	 orderDX	 orderDXCorrection	 orderDXLimited	 orderDXLimitedNotPrincipal	 
orderDXLimitedPrincipal	 orderInfo	 orgColumns	 orgValueCounts	 os	 pd	 plt	 preDX	 problem_list	 
raceColumns	 re	 recentDemo	 recentSocial	 social	 sus	 suscColumns	 susceptibilities	 suscmapping	 
sys	 target	 


In [1]:
%store

In [2]:
mldataset

NameError: name 'mldataset' is not defined